In [1]:
import re
import nltk


from collections import Counter, OrderedDict
from sklearn.base import TransformerMixin,BaseEstimator
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.util import ngrams
import numpy as np
import hashlib
import math
import pandas as pd 
import matplotlib.pyplot as plt
import random
import string
from nltk.tokenize import WordPunctTokenizer

from sklearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score, precision_score, recall_score,classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.svm import SVC
import spacy
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn import preprocessing

from bs4 import BeautifulSoup
from spacy.lang.en import English

from pylab import barh,plot,yticks,show,grid,xlabel,figure,cla,close
from sklearn.linear_model import LinearRegression

# Regex

. - matches any character, except newline.

\d, \s \S - match digit, match whitespace, not whitespace.

\b, \B - word, not word boundary.

[xyz] - matches x, y or z.

[^xyz] - matches anything that is not x, y or z.

[x-z] - matches a character between x and z.

^xyz$ - ^ is the start of the string, $ is the end of the string.

\. - use escaping to match special characters.

\t, \n - matches tab and newline.

x* - matches 0 or more symbols x.

x+ - matches 1 or more symbols x.

x? - matches 0 or 1 symbol x.

.?, *?, +?, etc - represent non-greedy search.

x{5} - matches exactly 5 symbols x.

x{5,} - matches 5 or more symbols x.

x{5, 8} - matches between 5 and 8 symbols x.

xy|yz - matches xy or yz.

In [ ]:
#nltk.download('stopwords')
#stemmer = SnowballStemmer("english", ignore_stopwords=True)
#stems = [list(map(stemmer.stem, words))]

# Read the data

In [2]:
#Load the dataset
##there are two columns in the data set sentiment which is positive and negative and text columns this is movie review
#df = pd.read_csv('./datasets/twitter.csv')
df_train_nb = pd.read_csv('./data/covid19_data.csv')#, encoding='latin1')
#df.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1,inplace=True)
df_train_nb = df_train_nb[['class','title']]
df_train_nb.rename(columns={"class":"target", "title":"text"},inplace=True)
df_test= pd.read_csv('data/covid19_unlabelled_test.csv')

df_test = df_test[['title']]
df_test.rename(columns={"title":"text"},inplace=True)
df_test.head()
#NOTE: If the data set is so many choose part of that in order to accelerate the process of analysing
    ##docs = df.text[:200]
# Get the text
#docs = df['text']



###we can read from file and represent it as list as follows
#def file_to_list(file_name):
    #with open(file_name, 'r', encoding='utf-8') as f:
        #return [line.strip() for line in f.readlines()]
    
#X_train_pre = file_to_list('data/tweets_train_preprocessed.txt')
#X_dev_pre = file_to_list('data/tweets_dev_preprocessed.txt')
#X_test_pre = file_to_list('data/tweets_test_preprocessed.txt')

##############################################################Read a JSON file################################

#docs = []
#with open('./datasets/sample_data.json') as fp:
    #for line in fp:
        #entry = json.loads(line)
        #docs.append(entry['body'])
        
#print('I read {} documents'.format(len(docs)))

,text
0,American researcher Charles Lieber was arrest...
1,Did Trump Blame Obama for ‘Bad’ COVID-19 Tests...
2,Hundreds of sampoerna cigarette factory worke...
3,Dr. Megha Vyas from Pune died of COVID-19.
4,Jacob Rothschild owns a patent to coronavirus.


In [3]:
df_train_nb['target'].unique()

array(['FALSE', 'False', 'Correct Attribution', 'True', 'Explanatory',
       'false', 'Misleading', 'MISLEADING', 'Mostly True', 'No Evidence',
       'partly false', 'Mostly false', 'Partly false', 'PARTLY FALSE',
       'No evidence', 'Mostly False', 'misleading', 'Labeled Satire',
       'News', 'Partly False', 'Misleading/False', 'Mixture',
       'no evidence', 'MOSTLY FALSE', 'Misattributed', 'Unproven',
       'Miscaptioned', 'Fake', 'Half True', 'PARTLY TRUE', 'Not true',
       'Scam', "(Org. doesn't apply rating)", 'Partially false',
       'MOSTLY TRUE', 'Partly true', 'mislEADING', 'NO EVIDENCE',
       'half true', 'false and misleading', 'mostly false', nan,
       'HALF TRUE', 'Two Pinocchios', 'Suspicions', 'Partly FALSE',
       'PANTS ON FIRE', 'Correct', 'Misinformation / Conspiracy theory',
       'IN DISPUTE', 'HALF TRUTH', 'MiSLEADING', 'Partially correct',
       'Unlikely', 'Fake news', 'True but', 'Mostly true', 'Collections',
       'Mixed', 'Unverified', 'Pa

In [4]:
df_train = pd.read_csv('./data/df_10percTrue_with_noevidence.csv')
df_train.drop(["Unnamed: 0"], axis=1,inplace=True)
df_train.rename(columns={"label":"target", "title":"text"},inplace=True)
df_train

,text,target
0,A viral image suggests that cocaine kills cor...,f
1,Having sex every morning kills coronavirus.,f
2,Picture shows the Mecca totally empty.,f
3,Queen Elizabeth is shown in a quote card to b...,f
4,Official news: in Naples people have used Toc...,f
...,...,...
1755,Bill Gates patented the coronavirus.,t
1756,Vitamin D Supplements prevent health problems...,t
1757,Smoking may protect against COVID-19; scienti...,t
1758,Ibuprofen enhances the coronavirus.,t


In [5]:
###function define to transform target column to true and false
def clean_targ_col(df_):
    df_['target'] = df_['target'].replace({'False' : 'false',
    'FALSE' : 'false',
    'Misleading' : 'false',
    'MISLEADING' : 'false',
    'Mostly false' : 'false',
    'Partly false': 'false',
    'misleading' : 'false',
    'No evidence' : 'true',
    'Mostly False' : 'false',
    'Mixture': 'false',
    'True': 'true',
    'Explanatory': 'true',
    'No Evidence': 'true',
    'News': 'true',
    'PARTLY FALSE': 'false',
    'Unproven': 'false',
    'MOSTLY FALSE': 'false',
    'Partly False': 'false',
    'Miscaptioned': 'false',
    'partly false': 'false',
    'mostly false' : 'false',
    'Mostly True': 'true',
    'MOSTLY TRUE': 'true',
    'Misattributed': 'false',
    "(Org. doesn't apply rating)": 'false',
    'HALF TRUE': 'true',
    'Correct Attribution' :'true',
    'Partially false': 'false',
    'Labeled Satire': 'false',
    'Fake' : 'false',
    'NO EVIDENCE': 'true',
    'false' : 'false',
    'Two Pinocchios': 'false',
    'Scam' : 'false',
    'no evidence': 'true',
    'Half True': 'true',
    'PARTLY TRUE': 'true',
    'half true': 'true',
    'Correct': 'true',
    'mislEADING' : 'false',
    'Suspicions': 'false',
    'Not true' : 'false',
    'nan': 'false',
    'Partly FALSE': 'false',
    'Misleading/False' : 'false',
    'PANTS ON FIRE': 'false',
    'Partially true': 'true',
    'Mixed': 'false',
    'IN DISPUTE': 'false',
    'Unverified': 'false',
    'HALF TRUTH': 'true',
    'Collections': 'false',
    'Partially correct': 'true',
    'MiSLEADING' : 'false',
    'Mostly true': 'true',
    'True but': 'true',
    'false and misleading' : 'false',
    'Partly true': 'true',
    'Misinformation / Conspiracy theory' : 'false',
    'Unlikely' : 'false',
    'Fake news' : 'false'})
    return df_

In [6]:
df_train_clean = clean_targ_col(df_train)
df_train_clean

,text,target
0,A viral image suggests that cocaine kills cor...,f
1,Having sex every morning kills coronavirus.,f
2,Picture shows the Mecca totally empty.,f
3,Queen Elizabeth is shown in a quote card to b...,f
4,Official news: in Naples people have used Toc...,f
...,...,...
1755,Bill Gates patented the coronavirus.,t
1756,Vitamin D Supplements prevent health problems...,t
1757,Smoking may protect against COVID-19; scienti...,t
1758,Ibuprofen enhances the coronavirus.,t


# Encoding the target coulmns to 0 and 1

In [ ]:
# Encode the labels
#le = preprocessing.LabelEncoder()
#le.fit(df_train_clean['target'].values)
#df_train_clean['target'] = le.transform(df_train_clean['target'].values)


In [7]:
df_train_clean['target'] = df_train_clean['target'].replace({'f':0,'t':1})
df_train_clean

,text,target
0,A viral image suggests that cocaine kills cor...,0
1,Having sex every morning kills coronavirus.,0
2,Picture shows the Mecca totally empty.,0
3,Queen Elizabeth is shown in a quote card to b...,0
4,Official news: in Naples people have used Toc...,0
...,...,...
1755,Bill Gates patented the coronavirus.,1
1756,Vitamin D Supplements prevent health problems...,1
1757,Smoking may protect against COVID-19; scienti...,1
1758,Ibuprofen enhances the coronavirus.,1


In [8]:
df_nc_train = df_train_clean.dropna().copy()
df_nc_test = df_test.copy()

# Split raw data without cleaning to train and validation

In [9]:
# Split in train and validation
train_nc_df, validation_nc_df = train_test_split(df_nc_train, test_size=0.2, random_state=42)

In [10]:
validation_nc_df.target.unique()

array([0, 1])

# Cleaning the data

In [11]:
# Custom transformer to implement sentence cleaning
class TextCleanerTransformer(TransformerMixin):
    def __init__(self, tokenizer, stemmer, regex_list,
                 lower=True, remove_punct=True):
        self.tokenizer = tokenizer
        self.stemmer = stemmer
        self.regex_list = regex_list
        self.lower = lower
        self.remove_punct = remove_punct
        
    def transform(self, X, *_):
        X = list(map(self._clean_sentence, X))
        return X
    
    def _clean_sentence(self, sentence):
        
        # Replace given regexes
        for regex in self.regex_list:
            sentence = re.sub(regex[0],regex[1], sentence)
            
        # lowercase
        if self.lower:
            sentence = sentence.lower()

        # Split sentence into list of words
        words = self.tokenizer.tokenize(sentence)
            
        # Remove punctuation
        if self.remove_punct:
            words = list(filter(lambda x: x not in string.punctuation, words))

        # Stem words
        if self.stemmer:
            words = map(self.stemmer.stem, words)

        # Join list elements into string
        sentence = " ".join(words)
        
        return sentence
    
    def fit(self, *_):
        return self

# Cleaning the data by function not class

In [12]:

tokenizer = WordPunctTokenizer()

def remove_html_tags(text):
    soup = BeautifulSoup(text)
    return soup.get_text()

def remove_punct(text):
    #remove everything except words, digits and space
    text = re.sub(r'[^\w\s]','',text) 
        
    #regex often miss the underscore so let's remove that as well
    text = re.sub(r'\_','',text)
    text = text.lower()
    
    return text

def remove_stopwords(text, stopwords):
    tokens = tokenizer.tokenize(text)
    tokens = [tok.lower() for tok in tokens]
    if stopwords:
        tokens = [tok for tok in tokens if tok not in stopwords]
    tokens = [stemmer.stem(tok) for tok in tokens]

    text_processed = ' '.join(tokens)
    return text_processed



def preprocessing_(df):
    """
    Implement the three above functions in the respective order to remove html tags, punctuations and stopwords
    Hint: Use the apply function.
    
    """
    df_ = df.copy()
    
    #df_['text'] = df_['text'].apply(...).apply(...).apply(...)
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    df_['text'] = df_['text'].apply(remove_html_tags).apply(remove_punct).apply(remove_stopwords,stopwords=en_stopwords)
    return df_

In [13]:
# Initialize a tokenizer and a stemmer
tokenizer = WordPunctTokenizer()
stemmer = SnowballStemmer("english", ignore_stopwords=True)
#regex_list = [("<[^>]*>", "")]
regex_list = [("\#","")]

#cleaner = TextCleanerTransformer(tokenizer, stemmer, regex_list)
#docs = cleaner.transform(train_df.text.values)

# Define pipline and predict MultinomialNB()(Did not work)

In [ ]:

# Build the pipeline
#text_clf = Pipeline([('prep', TextCleanerTransformer(tokenizer, stemmer,regex_list)),
 #                  ('vect', CountVectorizer(stop_words='english', ngram_range=(1,2))),
  #                 ('tfidf', TfidfTransformer()),
   #                ('clf', MultinomialNB())])
# Train the classifier
##text_clf.fit(map(str, train_df['text'].values), train_df['sentiment'].values)
#text_clf.fit(map(str, train_nc_df['text'].values), train_nc_df['target'].values)

##predicted = text_clf.predict(map(str, validation_df['text'].values))
##np.mean(predicted == validation_df['sentiment'])

#predicted_val = text_clf.predict(map(str, validation_nc_df['text'].values))
#np.mean(predicted_val == validation_nc_df['target'])

# Define pipeline and predict RandomForrestcalssifier(I am not using this right now)

In [14]:
# Build the pipeline
text_clf = Pipeline([('prep', TextCleanerTransformer(tokenizer, stemmer, regex_list)),
                   ('vect', CountVectorizer(stop_words='english', ngram_range=(1,2))),
                   ('tfidf', TfidfTransformer()),
                   ('classifier', RandomForestClassifier(random_state = 42))])
# Train the classifier
##text_clf.fit(map(str, train_df['text'].values), train_df['sentiment'].values)
text_clf.fit(map(str, train_nc_df['text'].values), train_nc_df['target'].values)

##predicted = text_clf.predict(map(str, validation_df['text'].values))
##np.mean(predicted == validation_df['sentiment'])

predicted_val = text_clf.predict(map(str, validation_nc_df['text'].values))
np.mean(predicted_val == validation_nc_df['target'])

0.9034090909090909

# Check classification result

In [15]:
# check the results
##print(classification_report(y_dev, y_dev_pred))
print(classification_report(validation_nc_df['target'].values,predicted_val))

              precision    recall  f1-score   support

           0       0.91      0.99      0.95       315
           1       0.71      0.14      0.23        37

    accuracy                           0.90       352
   macro avg       0.81      0.56      0.59       352
weighted avg       0.89      0.90      0.87       352



# Prediction for the test set

In [16]:
predicted_test = text_clf.predict(map(str, df_nc_test['text'].values))

In [17]:
df_nc_test['predicted'] = predicted_test
df_nc_test

,text,predicted
0,American researcher Charles Lieber was arrest...,0
1,Did Trump Blame Obama for ‘Bad’ COVID-19 Tests...,1
2,Hundreds of sampoerna cigarette factory worke...,0
3,Dr. Megha Vyas from Pune died of COVID-19.,0
4,Jacob Rothschild owns a patent to coronavirus.,0
...,...,...
2087,President Emmanuel Macron was cheered by a cr...,0
2088,Five helicopters are going to spray the air w...,0
2089,Coronavirus was created in a lab in order to ...,0
2090,Video of a man dressed in white who recommend...,0


In [18]:
df_nc_test.predicted.unique()

array([0, 1])

In [19]:
df_nc_test['predicted']=df_nc_test['predicted'].replace({0:'false',1:'true'})
df_nc_test

,text,predicted
0,American researcher Charles Lieber was arrest...,false
1,Did Trump Blame Obama for ‘Bad’ COVID-19 Tests...,true
2,Hundreds of sampoerna cigarette factory worke...,false
3,Dr. Megha Vyas from Pune died of COVID-19.,false
4,Jacob Rothschild owns a patent to coronavirus.,false
...,...,...
2087,President Emmanuel Macron was cheered by a cr...,false
2088,Five helicopters are going to spray the air w...,false
2089,Coronavirus was created in a lab in order to ...,false
2090,Video of a man dressed in white who recommend...,false


In [20]:
#submission file
def create_submit(df):
    df_out=pd.DataFrame()
    df_out['class'] = df_nc_test['predicted']
    return df_out
submission = create_submit(df_nc_test)
submission.to_csv("submission2.csv",index=False)

In [ ]:
#Export 
#su.to_csv('./data/submission.csv')

# How to counts the number of occurrences of different tokens in a list and whole corpus

def build_vocabulary():
    vocabulary = Counter()

    for doc in docs:
        words = doc.split()
        vocabulary.update(words)
    
    return OrderedDict(vocabulary.most_common())
# turn into a list of tuples and get the first 20 items
list(build_vocabulary().items())[:20]

# Information Extraction from text

In [ ]:
#! python -m spacy download en_core_web_sm

In [21]:
# Here we are disabling the synctatic parser from pipeline to improve speed.
nlp = spacy.load('en_core_web_sm', disable=['parser'])
en_stopwords = nlp.Defaults.stop_words
###################################################Clean data with the functions not classes

In [22]:
df_train_nb = pd.read_csv('./data/covid19_data.csv')#, encoding='latin1')
#df.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1,inplace=True)
###df_train_nb = df_train_nb[['class','title']]
df_train_nb.rename(columns={"class":"target", "title":"text"},inplace=True)
df_test= pd.read_csv('data/covid19_unlabelled_test.csv')

#df_test = df_test[['title']]
df_test.rename(columns={"title":"text"},inplace=True)
df_test.head()
#NOTE: If the data set is so

,verifiedby,country,text,published_date,country1,country2,country3,country4,article_source,ref_source,source_title,content_text,category,lang
0,Agência Lupa,Brazil,American researcher Charles Lieber was arrest...,2020/04/06,Brazil,NaN,NaN,NaN,https://piaui.folha.uol.com.br/lupa/2020/04/06...,poynter,#Verificamos: É falso que pesquisador norte-am...,", “HOMEM QUE VENDEU O CORONA VIRUS NA CHINA, P...",NaN,pt
1,snopes,NaN,Did Trump Blame Obama for ‘Bad’ COVID-19 Tests...,2020-05-01 00:00:00,NaN,NaN,NaN,NaN,https://www.snopes.com/fact-check/trump-blame-...,snopes,Did Trump Blame Obama for 'Bad' COVID-19 Tests?,['U.S. President Donald Trump implied the Obam...,Trump and the Pandemic II,en
2,TEMPO,Indonesia,Hundreds of sampoerna cigarette factory worke...,2020/05/05,Indonesia,NaN,NaN,NaN,https://cekfakta.tempo.co/fakta/765/fakta-atau...,poynter,Tempo.co,Narasi bahwa ratusan pekerja pabrik rokok Samp...,NaN,id
3,FactCrescendo,India,Dr. Megha Vyas from Pune died of COVID-19.,2020/04/25,India,NaN,NaN,NaN,https://www.malayalam.factcrescendo.com/image-...,poynter,FACT CHECK: ചിത്രത്തില്‍ കാണുന്ന സ്ത്രി പൂനയില...,"The fact behind every news!, കോവിഡ്\u200c-19 ര...",NaN,ml
4,Full Fact,United Kingdom,Jacob Rothschild owns a patent to coronavirus.,2020/01/30,United Kingdom,NaN,NaN,NaN,fullfact.org/online/Rothschild-coronavirus/,poynter,NaN,NaN,NaN,tl


In [ ]:
#df_nc_train = df_train_clean.copy()
#df_nc_test = df_test.copy()

In [23]:
#ad features before cleaning the data
def Add_feature(df):
    """
    Implement the three above functions in the respective order to remove html tags, punctuations and stopwords
    Hint: Use the apply function.
    """
    df_ = df.copy()

    df_['length'] = df_['text'].map(len)# we test if this feature is ok or not by plotting in the next cell we can do it for all the new features if it is skew to right and left it is important otherwise not inmportant
    df_['words'] = df_['text'].str.split().map(len)
    stop_words = nltk.corpus.stopwords.words('english')
#df_clean['words_not_stopword'] = df_clean['text'].apply(lambda x: len([t for t in x.split() if t not in stop_words]))
#df_clean['commas'] = df_clean['text'].str.count(',')
#df_clean['upper'] = df_clean['text'].map(lambda x: map(str.isupper, x)).map(sum)
#df_clean['capitalized'] = df_clean['text'].map(lambda x: map(str.istitle, x)).map(sum)
#get the average word length
#df_clean['avg_word_length'] = df_clean['text'].apply(lambda x: np.mean([len(t) for t in x.split() if t not in stop_words]) if len([len(t) for t in x.split(' ') if t not in stop_words]) > 0 else 0)
#number of adjective
#df_clean['n_adjs'] = n_adjs
#df_clean['positive_emojis_count'] = count_emoji_matches(pos_patterns)
#df_clean['negative_emojis_count'] = count_emoji_matches(neg_patterns)
    df_['hasTitle'] = 0
    df_.loc[df_['source_title'].isna(),'hasTitle'] = 1
    df_['hasContent'] = 0
    df_.loc[df_['content_text'].isna(),'hasContent'] = 1
    df_['hasLang'] = 0
    df_.loc[df.lang.isna(),'hasLang'] = 1
    df_['hasCountry1'] = 0
    df_.loc[~df.lang.isna(),'hasCountry1'] = 1
    df_['hasCountry2'] = 0
    df_.loc[~df.lang.isna(),'hasCountry2'] = 1
    df_['hasCountry3'] = 0
    df_.loc[~df.lang.isna(),'hasCountry3'] = 1
    df_['hasCountry4'] = 0
    df_.loc[~df.lang.isna(),'hasCountry4'] = 1
    df_['num_countries'] = df_['hasCountry1']+df_['hasCountry2']+df_['hasCountry3']+df_['hasCountry4']
    return df_

df_nc_train = Add_feature(df_train_nb)
df_nc_test= Add_feature(df_test)

df_nc_train


,verifiedby,country,target,text,published_date,country1,country2,country3,country4,article_source,...,length,words,hasTitle,hasContent,hasLang,hasCountry1,hasCountry2,hasCountry3,hasCountry4,num_countries
0,Delfi Melo Detektorius (Lie Detector),Lithuania,FALSE,Claims that coronavirus is fake and Belarus i...,2020/05/11,Lithuania,NaN,NaN,NaN,https://www.delfi.lt/news/melo-detektorius/mel...,...,98,16,0,0,0,1,1,1,1,4
1,AfricaCheck,"United States, South Africa",False,Muammar Gaddafi predicted the current coronav...,2020/03/21,United States,South Africa,NaN,NaN,https://africacheck.org/fbcheck/no-gaddafi-did...,...,148,23,0,0,0,1,1,1,1,4
2,AFP,Hong Kong,False,Video shows quarantined people in a building ...,2020/02/17,Hong Kong,NaN,NaN,NaN,http://u.afp.com/QuarantineChina,...,58,9,0,0,0,1,1,1,1,4
3,FactCrescendo,India,False,"A video shows a massive explosion in Wuhan, C...",2020/02/17,India,NaN,NaN,NaN,https://english.factcrescendo.com/2020/02/17/t...,...,89,15,0,0,0,1,1,1,1,4
4,Newtral.es,"Spain, Peru, United States, Mexico",False,Chlorine dioxide cures the coronavirus.,2020/03/25,Spain,Peru,United States,Mexico,https://www.newtral.es/bulo-dioxido-de-cloro-c...,...,40,5,0,0,0,1,1,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5526,AFP,Argentina,MISLEADING,A video shows the Italian air force air show ...,2020/03/19,Argentina,NaN,NaN,NaN,https://factual.afp.com/el-video-del-espectacu...,...,95,17,0,0,0,1,1,1,1,4
5527,India Today,India,Mostly false,While coronavirus tests in other countries ar...,2020/03/25,India,NaN,NaN,NaN,https://www.indiatoday.in/fact-check/story/fac...,...,117,20,0,0,0,1,1,1,1,4
5528,TEMPO,Indonesia,Misleading,Vladimir Putin released 800 tigers andlions t...,2020/03/23,Indonesia,NaN,NaN,NaN,https://cekfakta.tempo.co/fakta/694/fakta-atau...,...,80,13,0,0,0,1,1,1,1,4
5529,Factcheck.Vlaanderen,Belgium,misleading,The cure for coronavirus can come from Antwer...,2020/05/05,Belgium,NaN,NaN,NaN,https://factcheck.vlaanderen/factcheck/komt-de...,...,65,10,0,0,0,1,1,1,1,4


In [25]:
df_clean = preprocessing_(df_nc_train)
df_clean_test=preprocessing_(df_nc_test)

# define docs as a list of sentences



In [27]:
df_clean.columns

Index(['verifiedby', 'country', 'target', 'text', 'published_date', 'country1',
       'country2', 'country3', 'country4', 'article_source', 'ref_source',
       'source_title', 'content_text', 'category', 'lang', 'length', 'words',
       'hasTitle', 'hasContent', 'hasLang', 'hasCountry1', 'hasCountry2',
       'hasCountry3', 'hasCountry4', 'num_countries'],
      dtype='object')

In [ ]:
def func_prep(fd_):    
    df_.drop(['verifiedby', 'country', 'published_date', 'country1',
       'country2', 'country3', 'country4', 'article_source', 'ref_source',
       'source_title', 'content_text', 'category', 'lang'], axis=1,inplace=True)
    return df_
df_clean = func_prep(df_clean)
df_clean_test = func_prep(df_clean_test)


In [ ]:
docs = list(df_clean.text.values)

In [ ]:
#We are going to use the function pipe to process all documents.
# One of the strenghts for SpaCy is the parallel processing using all your computer cores.
# In this step, SpaCy performs the NLP pipeline for all the docs, so it may take a while

docs = list(nlp.pipe(docs))

In [ ]:
docs[0].ents

#Matcher ans pass vocabulary to it
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab) # Pass the vocabulary object to Matcher.__init__()
pattern = [{'POS': 'ADV'},{'POS':'ADJ'}]
matcher.add('word',None,pattern)

##Extracting adjectives,
for i, doc in enumerate(docs[:400]):
    #print(i,doc)
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]  # the matched span
        print(i, start, end, span)
   

# Extracting locations

for i, doc in enumerate(docs[:400]):
    for e in doc.ents:
        if e.label_ == 'GPE':   
            print(i, e.text, e.start_char, e.end_char)

# Extracting entities

matcher = Matcher(nlp.vocab)
#
pattern = [{'ENT_TYPE':'GPE'}]
matcher.add('LOC', None, pattern)
#matcher.add(...)
most_common_ents_list = []
for doc in docs:
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]  # the matched span
        span_text = span.text  # the span as a string
        most_common_ents_list.append(span_text)
most_common_ents = Counter(most_common_ents_list).most_common(10)
most_common_ents

# Adding even more features

In [ ]:
#number of adjectives
#name_of__adjs = [[token for token in doc if token.pos_ == 'ADJ'] for doc in docs]
#n_adjs = [len(internal_list) for internal_list in name_of__adjs]
#####df['n_adjs'] = n_adjs
##Emoji

nlp = English()  # We only want the tokenizer, so no need to load a model
matcher = Matcher(nlp.vocab)

#pos_patterns = [{'TEXT':{"REGEX": r"\:\)"}}] #- For Positive emoji let's use ":)"
#neg_patterns = [{'TEXT':{"REGEX": r"\:\("}}] #- For Negative emoji let's use ":("

# Hint - Don't forget to escape the special character "(" and ")"
# YOUR CODE HERE
#raise NotImplementedError()

#def count_emoji_matches(pattern, docs = docs):
 #   matcher = Matcher(nlp.vocab)
  #  matcher.add("EMOJIS", None, pattern)
    
   # n_emojis = []
    #for doc in docs:
     #   matches = matcher(doc)
      #  emojis_count = len(matches)
       # for match in matches:
        #    emojis_count += 1
        #n_emojis.append(emojis_count)
            
  #  return n_emojis

#positive_emojis_count = sum(count_emoji_matches(pos_patterns))
#negative_emojis_count = sum(count_emoji_matches(neg_patterns))



##df['positive_emojis_count'] = count_emoji_matches(pos_patterns)
##df['negative_emojis_count'] = count_emoji_matches(neg_patterns)

# Adding more extra features

In [ ]:
df_clean['length'] = df_clean['text'].map(len)# we test if this feature is ok or not by plotting in the next cell we can do it for all the new features if it is skew to right and left it is important otherwise not inmportant
df_clean['words'] = df_clean['text'].str.split().map(len)
stop_words = nltk.corpus.stopwords.words('english')
#df_clean['words_not_stopword'] = df_clean['text'].apply(lambda x: len([t for t in x.split() if t not in stop_words]))
#df_clean['commas'] = df_clean['text'].str.count(',')
#df_clean['upper'] = df_clean['text'].map(lambda x: map(str.isupper, x)).map(sum)
#df_clean['capitalized'] = df_clean['text'].map(lambda x: map(str.istitle, x)).map(sum)
#get the average word length
#df_clean['avg_word_length'] = df_clean['text'].apply(lambda x: np.mean([len(t) for t in x.split() if t not in stop_words]) if len([len(t) for t in x.split(' ') if t not in stop_words]) > 0 else 0)
#number of adjective
#df_clean['n_adjs'] = n_adjs
#df_clean['positive_emojis_count'] = count_emoji_matches(pos_patterns)
#df_clean['negative_emojis_count'] = count_emoji_matches(neg_patterns)
df_['hasTitle'] = 0
df_.loc[df.source_title.isna(),'hasTitle'] = 1
df_['hasContent'] = 0
df_.loc[df.content_text.isna(),'hasContent'] = 1
df_['hasLang'] = 0
df_.loc[df.lang.isna(),'hasLang'] = 1
df_['hasCountry1'] = 0
df_.loc[~df.lang.isna(),'hasCountry1'] = 1
df_['hasCountry2'] = 0
df_.loc[~df.lang.isna(),'hasCountry2'] = 1
df_['hasCountry3'] = 0
df_.loc[~df.lang.isna(),'hasCountry3'] = 1
df_['hasCountry4'] = 0
df_.loc[~df.lang.isna(),'hasCountry4'] = 1
df_['num_countries'] = df_['hasCountry1']+df_['hasCountry2']+df_['hasCountry3']+df_['hasCountry4']
df_clean


In [ ]:
def preprocessing_(df):
    """
    Implement the three above functions in the respective order to remove html tags, punctuations and stopwords
    Hint: Use the apply function.
    """
    df_ = df.copy()
############################################## ADD Features ######################
    df_['hasTitle'] = 0
    df_.loc[df.source_title.isna(),'hasTitle'] = 1
    df_['hasContent'] = 0
    df_.loc[df.content_text.isna(),'hasContent'] = 1
    df_['hasLang'] = 0
    df_.loc[df.lang.isna(),'hasLang'] = 1
    df_['hasCountry1'] = 0
    df_.loc[~df.lang.isna(),'hasCountry1'] = 1
    df_['hasCountry2'] = 0
    df_.loc[~df.lang.isna(),'hasCountry2'] = 1
    df_['hasCountry3'] = 0
    df_.loc[~df.lang.isna(),'hasCountry3'] = 1
    df_['hasCountry4'] = 0
    df_.loc[~df.lang.isna(),'hasCountry4'] = 1
    df_['num_countries'] = df_['hasCountry1']+df_['hasCountry2']+df_['hasCountry3']+df_['hasCountry4']
    return df_

In [ ]:
 df_clean.isnull.sum()

In [ ]:
df_clean = df_clean.drop(['verifiedby', 'country', 'published_date', 'country1',
       'country2', 'country3', 'country4', 'article_source', 'ref_source',
       'source_title', 'content_text', 'category', 'lang'], axis=1,inplace=True)
df_clean_test = df_clean_test.drop(['verifiedby', 'country', 'target', 'text', 'published_date', 'country1',
       'country2', 'country3', 'country4', 'article_source', 'ref_source',
       'source_title', 'content_text', 'category', 'lang'], axis=1,inplace=True)

# Is this feature usefull?(by the plot the length is  useful)

In [ ]:
#length column
ax_list = df_clean.hist(column='length', by='target', bins=50,figsize=(12,4))
ax_list[0].set_xlim((0,300))
ax_list[1].set_xlim((0,300))

In [ ]:
ax_list = df_clean.hist(column='words', by='target', bins=50,figsize=(12,4))
ax_list[0].set_xlim((0,300))
ax_list[1].set_xlim((0,300))

# Class for choosing extra features

In [ ]:
class Selector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a column from the dataframe to perform additional transformations on
    """ 
    def __init__(self, key):
        self.key = key
        
    def fit(self, X, y=None):
        return self
    

class TextSelector(Selector):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on text columns in the data
    """
    def transform(self, X):
        return X[self.key]
    
    
class NumberSelector(Selector):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def transform(self, X):
        return X[[self.key]]
    
    

# Define a pipeline

In [ ]:
text = Pipeline([
                ('selector', TextSelector("text")),
                ('tfidf', TfidfVectorizer())
            ])

length =  Pipeline([
                ('selector', NumberSelector("length")),
                ('standard', StandardScaler())
            ])

words =  Pipeline([
                ('selector', NumberSelector(key='words')),
                ('standard', StandardScaler())
            ])
words_not_stopword =  Pipeline([
                ('selector', NumberSelector(key='words_not_stopword')),
                ('standard', StandardScaler())
            ])
#avg_word_length =  Pipeline([
 #               ('selector', NumberSelector(key='avg_word_length')),
  #              ('standard', StandardScaler())
   #         ])
commas =  Pipeline([
                ('selector', NumberSelector(key='commas')),
                ('standard', StandardScaler()),
            ])
upper =  Pipeline([
                ('selector', NumberSelector(key='upper')),
                ('standard', StandardScaler()),
            ])
capitalized =  Pipeline([
                ('selector', NumberSelector(key='capitalized')),
                ('standard', StandardScaler()),
            ])

#n_adjs = Pipeline(['selector', NumberSelector(key='n_adjs'),
                  # ('standard', StandardScaler()),
                 # ])

positive_emojis_count  = Pipeline([
                ('selector', NumberSelector("positive_emojis_count"))
            ])
negative_emojis_count = Pipeline([
               ('selector', NumberSelector("negative_emojis_count"))
            ])


feats = FeatureUnion([('text', text), 
                      ('length', length),
                      ('words', words),
                      ('words_not_stopword', words_not_stopword),
                      #('avg_word_length', avg_word_length),
                      ('commas', commas),
                     ('upper', upper),
                     ('capitalized', capitalized),
                     #('n_adjs',n_adjs),
                      ('positive_emojis_count',positive_emojis_count),
                      ('negative_emojis_count',negative_emojis_count),
                     ])

feature_processing = Pipeline([('feats', feats)])

In [ ]:
#df_clean = df_clean.reset_index(drop = True)
#df_clean

# Split train and test of clean data and fitting the model and predicting

In [ ]:
# Split in train and validation
train_cl_data, test_cl_data = train_test_split(df_clean, test_size=0.2, random_state=42)



In [ ]:
train_cl_data.isnull().sum()

In [ ]:
pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
    #('classifier', LinearRegression()),
])

pipeline.fit(train_cl_data, train_cl_data.target)

preds = pipeline.predict(test_cl_data)
np.mean(preds == test_cl_data.target)

# How to reduce the features and choose the more important one
1- Feature Selection from chi-squared
2-Feature selection with SVD and PCA

# # The following will plot the  most dependent features from the chi-squared values.

In [ ]:
nltk.download("stopwords") 
stopword_list =list(nltk.corpus.stopwords.words("english"))

In [ ]:
#preprocess to run the following cell about chi_values
#df is already clean data
#Encode the lables y_train and y_dev
vec = TfidfVectorizer(ngram_range=(1,2))
X_train_vec = vec.fit_transform(train_cl_data.text)
X_test_vec = vec.transform(test_cl_data.text)


In [ ]:
chi_values, p_values = chi2(X_train_vec, train_cl_data.target.values)
print(chi_values, p_values)

In [ ]:
feature_names = vec.get_feature_names()

cla()   # Clear axis
close() # Close a figure window

figure(figsize=(12,10))
zipped_chi_squared = zip(feature_names, chi_values)
sorted_chi_values = sorted(zipped_chi_squared, key=lambda x:x[1]) 
top_chi_values = list(zip(*sorted_chi_values[-30:]))

x = range(len(top_chi_values[1]))
labels = top_chi_values[0]
barh(x, list(top_chi_values)[1], align='center', alpha=.2, color='g')
yticks(x, labels)
xlabel('$\chi^2$')
show()

# Ch2 choose best features

In [ ]:
ch2 = SelectKBest(chi2, k=10)
ch2.fit(X_train_vec, train_cl_data.target)

most_important_features = [feature_names[i] for i in ch2.get_support(indices=True)]
for f in most_important_features:
    print(f)

# Prediction

In [ ]:
X_train_chi = ch2.transform(X_train_vec)
X_test_chi = ch2.transform(X_test_vec)

clf = MultinomialNB()
%timeit clf.fit(X_train_chi, train_cl_data.target)

y_pred = clf.predict(X_test_chi)